In [1]:
print("hello world")

hello world


In [ ]:
%cd ..

import wandb
import torch, wandb, os
import torch.optim as optim

from src.RandmanFunctions import RandmanConfig, split_and_load
from src.Models import RandmanSNNConfig, RandmanSNN, spike_regularized_cross_entropy
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.EvolutionAlgorithms.PseudoPSO import PPSOModel, PPSOModelWithPooling
from src.SurrogateGD.VanillaSGD import SGDModel
from src.Training import train_loop_snn
from src.Utilities import init_result_csv, set_seed
from src.LandscapeAnalysis.LossSurfacePlotter import LossSurfacePlotter

device = 'cuda'
loss_fn_spk = lambda logits, y, spikes: spike_regularized_cross_entropy(logits, y, spikes, lambda_reg=1e-3)
config_id = 1
snn_config = RandmanSNNConfig.lookup_by_id(config_id)

@torch.no_grad()
def train_snn():
    run_name = "wenqi-pipeline-test"
    config = {  # Dataset:
        "nb_input": 3,
        "nb_output": 2,
        "nb_steps": 50,
        "nb_data_samples": 1000,
        "dim_manifold": 1,
        "alpha": 3.0,
        # SNN:
        "nb_hidden": 100,
        "learn_beta": False,
        # Evolution Strategy:
        "nb_model_samples": 20,
        "mirror": True,
        # Training:
        "std": 0.1,
        "epochs": 100,
        "batch_size": 256,
        "method": "es",
        # Optimization:
        "loss": "loss_fn",
        "optimizer": "Adam",
        "lr": 0.01,
        "regularization": "none",
    }
    with wandb.init(
        entity="DarwinNeuron", project="Test", name=run_name, config=config
    ) as run:
        # update current run_name
        keys = ["method", "std", "batch_size", "lr", "nb_model_samples", "loss", "nb_input"]
        sorted_items = [f"{getattr(run.config, k)}" for k in sorted(keys)]
        run.name = "-".join(sorted_items)
        
        # setting up local csv recording (optional)
        # result_path, _, _ = init_result_csv(dict(run.config), run.project)

        # my_model = ESModel(
        #     RandmanSNN,
        #     run.config.nb_input,
        #     run.config.nb_hidden,
        #     run.config.nb_output,
        #     run.config.learn_beta, 
        #     sample_size=run.config.nb_model_samples,
        #     param_std=run.config.std,
        #     Optimizer=optim.Adam,
        #     lr=run.config.lr,
        #     device=device,
        #     mirror=run.config.mirror,
        # )
        
        my_model = ESModel(
            RandmanSNN,
            run.config.nb_input,
            run.config.nb_output,
            snn_config,
            sample_size=run.config.nb_model_samples,
            param_std=run.config.std,
            Optimizer=optim.Adam,
            lr=run.config.lr,
            device=device,
            mirror=run.config.mirror,
        )

        # load dataset
        train_loader, val_loader = split_and_load(
            RandmanConfig(
                nb_classes=run.config.nb_output,
                nb_units=run.config.nb_input,
                nb_steps=run.config.nb_steps,
                nb_samples=run.config.nb_data_samples,
                dim_manifold=run.config.dim_manifold,
                alpha=run.config.alpha,
            ).read_dataset(),
            run.config.batch_size,
        )
        
        # loss surface plotter
        plotter_dir = f"results/{run.project}/runs/{run.id}/"
        os.makedirs(plotter_dir, exist_ok=True)
        loss_plotter = LossSurfacePlotter(plotter_dir+"illuminated_loss_surface.npz")

        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(my_model, train_loader, val_loader, loss_fn_spk, device, run, epoch, loss_plotter=None)

train_snn()

/scratch/wx2178/SNN/DarwinNeuron


/scratch/wx2178/.conda/envs/cns/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wx2178 (RNNFNNneuron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


filepath data/randman/5c40df6959bf49d1a4c55cb2a197c31b.pt
Epoch 0
-------------------------------
batch 0, loss: 0.693147, accuracy: 50.0%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 1, loss: 0.693147, accuracy: 48.4%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 2, loss: 0.693147, accuracy: 55.5%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 3, loss: 0.693147, accuracy: 50.0%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 4, loss: 0.693147, accuracy: 47.7%
Accuracy: 45.0%, Avg loss: 0.693147 

Epoch 1
-------------------------------
batch 0, loss: 0.693147, accuracy: 51.6%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 1, loss: 0.693147, accuracy: 52.3%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 2, loss: 0.693147, accuracy: 46.1%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 3, loss: 0.693147, accuracy: 49.6%
Accuracy: 45.0%, Avg loss: 0.693147 

batch 4, loss: 0.693147, accuracy: 54.0%
Accuracy: 45.0%, Avg loss: 0.693147 

Epoch 2
-------------------------------
batch 0, loss: 0.693147, accurac

In [ ]:
import wandb
run = wandb.init(
    entity="DarwinNeuron",
    project="Test",
    name="wandb-connection-diag",
    config={"diag": True},
)